# W266 Final Project Code
# Amazon Product Review Aspect-Based Sentiment
## Jennifer Mahle and Joanna Wang (Sections 3 and 1, respectively) 


In [2]:
import os
import pandas as pd

dataset = "Electronics_5.json"

if os.path.isfile(dataset):
    df = pd.read_json("Electronics_5.json", lines=True)
else:
    url = r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_10.json.gz"
    df = pd.read_json(url, compression='gzip', lines=True)

display(df.head(10))
display(df.tail(10))
df.shape
no_NA_reviews = df.dropna(subset=['reviewText'])
no_NA_reviews.shape
no_NA_style = df.dropna(subset=['style'])
no_NA_style.shape

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN
5,4,NaN,True,"06 5, 2013",A3IYSOTP3HA77N,0380709473,{'Format:': ' Kindle Edition'},B. Marks,I read this probably 50 years ago in my youth ...,Above average mystery,1370390400,NaN
6,5,NaN,True,"06 27, 2016",A11SXV34PZUQ5E,0380709473,{'Format:': ' Kindle Edition'},Tom C.,I read every Perry mason book voraciously. Fin...,Lam is cool!,1466985600,NaN
7,5,NaN,True,"07 30, 2015",A2AUQM1HT2D5T8,0380709473,{'Format:': ' Kindle Edition'},ema,I love this series of Bertha and Lamb.. Great...,Five Stars,1438214400,NaN
8,5,NaN,True,"02 16, 2015",A3UD8JRWLX6SRX,0380709473,{'Format:': ' Paperback'},Michael O.,Great read!,Five Stars,1424044800,NaN
9,4,NaN,False,"11 21, 2013",A3MV1KKHX51FYT,0380709473,{'Format:': ' Paperback'},Acute Observer,"Crows Can't Count, A.A. Fair\n\nMr. Harry Shar...",A Fast and Far Moving Adventure,1384992000,NaN


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
6739580,5,NaN,True,"07 25, 2017",A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800,NaN
6739581,1,NaN,True,"04 4, 2017",A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000,NaN
6739582,3,NaN,True,"07 8, 2017",A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000,NaN
6739583,2,NaN,True,"05 21, 2017",A12E1JGKV0ETAB,B01HJH42KU,NaN,John Adams,I liked the length and the product at first bu...,Lost ability to connect.,1495324800,NaN
6739584,3,NaN,True,"06 26, 2017",A1HKXEX8BEQC2E,B01HJH40WU,NaN,Dasha stephens,not holding up over time :(,not holding up over time :(,1498435200,NaN
6739585,4,NaN,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,NaN,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,NaN
6739586,4,NaN,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,NaN,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,NaN
6739587,5,2,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,NaN
6739588,5,2,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,NaN,Faith,Works well.,Five Stars,1480377600,NaN
6739589,5,NaN,True,"03 31, 2017",A1FGCIRPRNZWD5,B01HJF704M,NaN,Brando,I have it plugged into a usb extension on my g...,Works well enough..,1490918400,NaN


(3547420, 12)

In [3]:
display(df.tail(10))

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
6739580,5,NaN,True,"07 25, 2017",A1OOVLE2KZ6KGA,B01HJCN1EI,NaN,Puddzee,These are my favorite charging cords for a few...,Worth the price.,1500940800,NaN
6739581,1,NaN,True,"04 4, 2017",A77K1B31UAQ29,B01HJCN1EI,NaN,addictedtoreading,"Update....after 2 months of gentle use, cable ...",UPDATE...BREAKS AND SLOW CHARGING,1491264000,NaN
6739582,3,NaN,True,"07 8, 2017",A2SVXUVUAWUDK2,B01HJH42KU,NaN,Andrew,These are okay. The connection becomes very if...,Hope this makes sense. You'd understand if you...,1499472000,NaN
6739583,2,NaN,True,"05 21, 2017",A12E1JGKV0ETAB,B01HJH42KU,NaN,John Adams,I liked the length and the product at first bu...,Lost ability to connect.,1495324800,NaN
6739584,3,NaN,True,"06 26, 2017",A1HKXEX8BEQC2E,B01HJH40WU,NaN,Dasha stephens,not holding up over time :(,not holding up over time :(,1498435200,NaN
6739585,4,NaN,True,"03 21, 2017",A33MAQA919J2V8,B01HJH40WU,NaN,Kurt Wurm,"These seem like quality USB cables, time will ...",Four Stars,1490054400,NaN
6739586,4,NaN,True,"01 9, 2017",A1AKHSCPD1BHM4,B01HJH40WU,NaN,C.L Momof3,"Works great, love the longer cord. As with any...",Nice long cord,1483920000,NaN
6739587,5,2,True,"12 1, 2016",A2HUZO7MQAY5I2,B01HJH40WU,NaN,michael clontz,"Ok here is an odd thing that happened to me, I...",Not the correct product as linked in the sale.,1480550400,NaN
6739588,5,2,True,"11 29, 2016",AJJ7VX2L91X2W,B01HJH40WU,NaN,Faith,Works well.,Five Stars,1480377600,NaN
6739589,5,NaN,True,"03 31, 2017",A1FGCIRPRNZWD5,B01HJF704M,NaN,Brando,I have it plugged into a usb extension on my g...,Works well enough..,1490918400,NaN


In [4]:
no_NA_reviews = df.dropna(subset=['reviewText'])
no_NA_reviews.shape

(6738237, 12)

In [5]:
no_NA_style = df.dropna(subset=['style'])
no_NA_style.shape

(3547420, 12)

In [8]:
print(df["reviewText"].iloc[500])

Bought it to hang my 43" Samsung Plasma. I really like the 24" reach. Lots of flexibility. very strong. Been using it for over half the year and hasn't shown signs of sag.

Only problem I have was that the mounting plate blocked the power cord from the TV! Therefore I had to plug the cord (with some jigging around and loosening of screws) in first before mounting. One star off for this oddity. Or maybe it's a Samsung thing?


In [10]:
# Remove ## from lines starting with ! and run them the first time to install necessary packages 

##%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn


  Could not find a version that satisfies the requirement tensorflow-gpu (from versions: )
No matching distribution found for tensorflow-gpu
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 92kB 2.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/2f/5b/2cc2b9285e8b2ca8d2c1e4a2cbf1b12d70a2488ea78170de1909bca725f2/numpy-1.18.1-cp37-cp37m-macosx_10_9_x86_64.whl
    100% |████████████████████████████████| 1.3MB 10.5MB/s ta 0:00:01
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/Library/Python/3.7'
Consider using the `--user` option or check the permissions.

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 225kB 7.0MB/s ta 0:00:01
  Using cached https:/

In [4]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

/Users/joannawang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/joannawang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/joannawang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/joannawang/opt/anaconda3/lib/python3.7/site-pack

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
#split the data into training and testing data, using "overall" as the target variable
y=no_NA_reviews.overall
x=no_NA_reviews.drop('overall',axis=1)


In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

logging.set_verbosity(logging.ERROR)

#create embeddings on the training data 
message_embeddings = embed(x_train.reviewText)

In [37]:
print("Training X Shape", x_train.shape)
print("Testing X Shape", x_test.shape)

Training X Shape (5390589, 11)
Testing X Shape (1347648, 11)
